In [1]:
from utz.imports import *

In [ ]:
cache = '/Volumes/T7/citibike'
env['AWS_PROFILE'] = 'rac'

In [2]:
Bucket = 'tripdata'

In [3]:
from boto3 import client
from botocore import UNSIGNED
from botocore.client import Config
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [4]:
resp = s3.list_objects_v2(Bucket=Bucket)
zips = pd.DataFrame(resp['Contents'])
zips

,Key,LastModified,ETag,Size,StorageClass
0,201306-citibike-tripdata.zip,2018-04-30 13:18:55+00:00,"""b520a12de58eea58a3586f89bfcfbd9d-2""",16785103,STANDARD
1,201307-201402-citibike-tripdata.zip,2017-01-18 22:23:25+00:00,"""7b3b260b2ab2e5349320121d04bd821c-22""",178262576,STANDARD
2,201307-citibike-tripdata.zip,2017-01-18 22:23:27+00:00,"""dd3e6fd5f91715b31eae72868086c08c-4""",27074629,STANDARD
3,201308-citibike-tripdata.zip,2017-01-18 22:23:27+00:00,"""2f661063576734f614b9f1d6bba0ec59-4""",32090869,STANDARD
4,201309-citibike-tripdata.zip,2017-01-18 22:23:27+00:00,"""a42f947db7bd14e423a7dbfbb11596a1-4""",33155593,STANDARD
...,...,...,...,...,...
144,JC-202005-citibike-tripdata.csv.zip,2020-06-05 16:09:40+00:00,"""a8a53f2ae57b29a9fe287a7e666a8c74""",655725,STANDARD
145,JC-202006-citibike-tripdata.csv.zip,2020-07-06 00:37:53+00:00,"""ebaaa7c1eb9467d04a7748feb6fc4f5b""",968484,STANDARD
146,JC-202007-citibike-tripdata.csv.zip,2020-08-12 02:13:32+00:00,"""4519a964d5b66156f9e661b7e78906ac""",1017946,STANDARD
147,JC-202008-citibike-tripdata.csv.zip,2020-09-04 19:37:33+00:00,"""bc110ea6ded190e5625213e05dce31fb""",1137151,STANDARD


In [5]:
rgx = r'^(?P<JC>JC-)?(?P<year>\d{4})(?P<month>\d{2})-citibike-tripdata(?P<csv>\.csv)?(?P<zip>\.zip)?$'

In [6]:
from io import BytesIO
from zipfile import ZipFile

In [19]:
def to_parquet(zip_path, error='warn'):
    name = basename(zip_path)
    m = match(rgx, name)
    if not m:
        msg = f'Unrecognized key: {name}'
        if error == 'warn':
            print(msg)
            return
        else:
            raise Exception(msg)
    assert name.endswith('.zip'), name
    base = splitext(zip_path)[0]
    if base.endswith('.csv'):
        base = splitext(base)[0]

    pqt_path = f'{base}.parquet'
    if not exists(pqt_path):
        z = ZipFile(zip_path)
        names = z.namelist()
        print(f'{name}: zip names: {names}')
        [ name ] = [ f for f in names if f.endswith('.csv') and not f.startswith('_') ]
        with z.open(name,'r') as i:
            df = pd.read_csv(i)
            df.to_parquet(pqt_path)

In [8]:
zips = sorted(glob(f'{cache}/*.zip'))
[ basename(z) for z in zips ]

['201306-citibike-tripdata.zip',
 '201307-201402-citibike-tripdata.zip',
 '201307-citibike-tripdata.zip',
 '201308-citibike-tripdata.zip',
 '201309-citibike-tripdata.zip',
 '201310-citibike-tripdata.zip',
 '201311-citibike-tripdata.zip',
 '201312-citibike-tripdata.zip',
 '201401-citibike-tripdata.zip',
 '201402-citibike-tripdata.zip',
 '201403-citibike-tripdata.zip',
 '201404-citibike-tripdata.zip',
 '201405-citibike-tripdata.zip',
 '201406-citibike-tripdata.zip',
 '201407-citibike-tripdata.zip',
 '201408-citibike-tripdata.zip',
 '201409-citibike-tripdata.zip',
 '201410-citibike-tripdata.zip',
 '201411-citibike-tripdata.zip',
 '201412-citibike-tripdata.zip',
 '201501-citibike-tripdata.zip',
 '201502-citibike-tripdata.zip',
 '201503-citibike-tripdata.zip',
 '201504-citibike-tripdata.zip',
 '201505-citibike-tripdata.zip',
 '201506-citibike-tripdata.zip',
 '201507-citibike-tripdata.zip',
 '201508-citibike-tripdata.zip',
 '201509-citibike-tripdata.zip',
 '201510-citibike-tripdata.zip',
 '2

In [11]:
from joblib import delayed, Parallel
parallel = Parallel(n_jobs=cpu_count())

In [14]:
zips

['/Volumes/T7/citibike/201306-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201307-201402-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201307-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201308-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201309-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201310-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201311-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201312-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201401-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201402-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201403-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201404-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201405-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201406-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201407-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201408-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201409-citibike-tripdata.zip',
 '/Volumes/T7/citibike/201410-citibike-tripdata.zip',
 '/Volumes/T7/citibik

In [20]:
parallel(delayed(to_parquet)(f) for f in zips); None

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [63]:
def parse_year(df):
    year = df['birth year']
    #[idx] = df.index
    if year == '\\N':
        return nan
    else:
        year = float(year)
        if isna(year):
            return nan
        elif floor(year) != year:
            raise Exception(f'Invalid year: {year}, {df}')
        else:
            return int(year)

In [21]:
import dask.dataframe as dd

In [22]:
dfs = dd.read_parquet(f'{cache}/*.parquet')
dfs

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
npartitions=146,,,,,,,,,,,,,,,
,int64,object,object,float64,object,float64,float64,float64,object,float64,float64,int64,object,float64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [23]:
len(dfs)

105427227

In [40]:
from math import floor

In [62]:
import dask
dask.config.set(scheduler='processes')

In [65]:
yrs = dfs.apply(parse_year, axis=1, meta=('birth year', 'int64'))

In [66]:
yrs.compute()

AttributeError: Can't get attribute 'parse_year' on <module '__main__' (built-in)>

Traceback
---------
  File "/Users/ryan/.pyenv/versions/3.8.5/lib/python3.8/site-packages/dask/local.py", line 221, in execute_task
    task, data = loads(task_info)


In [28]:
years = dfs['birth year'].value_counts().compute()

In [32]:
years.values

array([5544308, 3732539, 3518454, ...,       1,       1,       1])

In [35]:
yrs = years.index.to_series()
yrs[yrs.apply(type) == str]

\N        \N
1983    1983
1985    1985
1984    1984
1982    1982
1981    1981
1986    1986
1980    1980
1987    1987
1988    1988
1979    1979
1978    1978
1989    1989
1977    1977
1976    1976
1974    1974
1970    1970
1990    1990
1975    1975
1969    1969
1972    1972
1973    1973
1971    1971
1968    1968
1967    1967
1966    1966
1964    1964
1963    1963
1965    1965
1962    1962
1960    1960
1991    1991
1961    1961
1959    1959
1958    1958
1957    1957
1956    1956
1992    1992
1955    1955
1954    1954
1953    1953
1993    1993
1951    1951
1952    1952
1994    1994
1950    1950
1948    1948
1949    1949
1995    1995
1947    1947
1946    1946
1996    1996
1945    1945
1997    1997
1944    1944
1942    1942
1941    1941
1943    1943
1940    1940
1900    1900
1938    1938
1998    1998
1939    1939
1935    1935
1936    1936
1934    1934
1899    1899
1901    1901
1937    1937
1932    1932
1933    1933
1922    1922
1926    1926
1910    1910
1921    1921
1917    1917
1924    1924

In [31]:
years.apply(type).value_counts()

<class 'int'>    17942
Name: birth year, dtype: int64

In [25]:
dfs.starttime.head().dt.year

AttributeError: Can only use .dt accessor with datetimelike values

In [107]:
csv = pd.read_csv(csvs[0])
csv

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1010,2013-09-01 00:00:02,2013-09-01 00:16:52,254,W 11 St & 6 Ave,40.735324,-73.998004,147,Greenwich St & Warren St,40.715422,-74.011220,15014,Subscriber,1974,1
1,1443,2013-09-01 00:00:09,2013-09-01 00:24:12,151,Cleveland Pl & Spring St,40.721816,-73.997203,497,E 17 St & Broadway,40.737050,-73.990093,19393,Customer,\N,0
2,1387,2013-09-01 00:00:16,2013-09-01 00:23:23,352,W 56 St & 6 Ave,40.763406,-73.977225,405,Washington St & Gansevoort St,40.739323,-74.008119,16160,Subscriber,1992,1
3,405,2013-09-01 00:00:18,2013-09-01 00:07:03,490,8 Ave & W 33 St,40.751551,-73.993934,459,W 20 St & 11 Ave,40.746745,-74.007756,14997,Subscriber,1973,1
4,270,2013-09-01 00:00:20,2013-09-01 00:04:50,236,St Marks Pl & 2 Ave,40.728419,-73.987140,393,E 5 St & Avenue C,40.722992,-73.979955,19609,Subscriber,1984,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034354,93,2013-09-30 23:59:16,2013-10-01 00:00:49,360,William St & Pine St,40.707179,-74.008873,195,Liberty St & Broadway,40.709056,-74.010434,20594,Subscriber,1989,1
1034355,363,2013-09-30 23:59:21,2013-10-01 00:05:24,2003,1 Ave & E 18 St,40.734161,-73.980243,459,W 20 St & 11 Ave,40.746745,-74.007756,15213,Customer,\N,0
1034356,562,2013-09-30 23:59:30,2013-10-01 00:08:52,293,Lafayette St & E 8 St,40.730287,-73.990765,463,9 Ave & W 16 St,40.742065,-74.004432,17938,Subscriber,1967,1
1034357,1237,2013-09-30 23:59:40,2013-10-01 00:20:17,478,11 Ave & W 41 St,40.760301,-73.998842,318,E 43 St & Vanderbilt Ave,40.753202,-73.977987,15931,Customer,\N,0


In [114]:
sub('\.csv$', '.parquet', csvs[0])

'/Volumes/T7/citibike/201309-citibike-tripdata.parquet'

In [112]:
csvs[0].replace('.csv','.parquet')

'/Volumes/T7/citibike/201309-citibike-tripdata.parquet'

In [115]:
csv.to_parquet(sub('\.csv$', '.parquet', csvs[0]))

In [40]:
def get_month(year=None, month=None, key=None, jc=False, cache='citibike', overwrite=False, dask=True):
    if year is not None and month is not None:
        assert key is None
        key = f'{year}{"%02d" % int(month)}-citibike-tripdata.zip'
        if jc: key = f'JC-{key}'
    elif key is not None:
        assert year is None
        assert month is None

    cache_path = f'{cache}/{key}'
    if not exists(cache_path) or overwrite:
        with open(cache_path,'wb') as f:
            resp = s3.get_object(Bucket=Bucket, Key=key)
            f.write(resp['Body'].read())

    if dask:
        from dask import dataframe as dd
        read_csv = dd.read_csv
    else:
        read_csv = pd.read_csv

    with open(cache_path,'rb') as f:
        z = ZipFile(BytesIO(body))
        [ filename ] = [ 
            filename
            for filename in z.namelist
            if match(rgx, filename)
        ]
        with z.open(filename,'rb') as i,
        csv_str = decode(z.read(m.string))
        print(type(csv_bytes))
        df = read_csv(csv_str)

    return df

In [41]:
df_201306 = get_month(2013, 6)
df_201306

NameError: name 'decode' is not defined